In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fungsi keanggotaan trapesium
def trapezoid(x, a, b, c, d):
    if x < a or x > d:
        return 0.0
    elif a <= x < b:
        return (x - a) / (b - a)
    elif b <= x <= c:
        return 1.0
    elif c < x <= d:
        return (d - x) / (d - c)
    else:
        return 0.0

# Fungsi keanggotaan segitiga
def triangle(x, a, b, c):
    if x <= a or x >= c:
        return 0.0
    elif a < x <= b:
        return (x - a) / (b - a)
    elif b < x < c:
        return (c - x) / (c - b)
    else:
        return 0.0

# Fuzzifikasi nilai IPK
def fuzzifikasi_ipk(ipk):
    rendah = trapezoid(ipk, 0, 0, 2.75, 2.75)
    sedang = triangle(ipk, 2.5, 3.0, 3.5)
    tinggi = trapezoid(ipk, 3.25, 4.0, 4.0, 4.0)
    return {'rendah': rendah, 'sedang': sedang, 'tinggi': tinggi}

# Fuzzifikasi jumlah publikasi
def fuzzifikasi_publikasi(pub):
    sedikit = trapezoid(pub, 0, 0, 2, 2)
    cukup = triangle(pub, 1, 2.5, 4)
    banyak = trapezoid(pub, 3, 10, 10, 10)
    return {'sedikit': sedikit, 'cukup': cukup, 'banyak': banyak}

# Inferensi aturan menggunakan metode Mamdani (MIN)
def inferensi(ipk, publikasi):
    rules = []
    μ_ipk = fuzzifikasi_ipk(ipk)
    μ_pub = fuzzifikasi_publikasi(publikasi)

    rules.append(('cumlaude', min(μ_ipk['tinggi'], μ_pub['banyak'])))
    rules.append(('sangat', min(μ_ipk['tinggi'], μ_pub['cukup'])))
    rules.append(('sangat', min(μ_ipk['sedang'], μ_pub['cukup'])))
    rules.append(('memuaskan', min(μ_ipk['sedang'], μ_pub['sedikit'])))
    rules.append(('memuaskan', μ_ipk['rendah']))
    return rules

# Defuzzifikasi dengan metode centroid
X_UNIVERSE = np.linspace(0, 100, 1000)
MU_MEMUASKAN = np.array([trapezoid(val, 0, 0, 60, 60) for val in X_UNIVERSE])
MU_SANGAT = np.array([triangle(val, 40, 60, 80) for val in X_UNIVERSE])
MU_CUMLAUDE = np.array([trapezoid(val, 70, 100, 100, 100) for val in X_UNIVERSE])

def defuzzifikasi_with_plot_data(rules):
    μ_agg = np.zeros_like(X_UNIVERSE)
    for predikat, α in rules:
        if α == 0.0: continue
        if predikat == 'memuaskan': μ_implikasi = np.minimum(α, MU_MEMUASKAN)
        elif predikat == 'sangat': μ_implikasi = np.minimum(α, MU_SANGAT)
        elif predikat == 'cumlaude': μ_implikasi = np.minimum(α, MU_CUMLAUDE)
        μ_agg = np.maximum(μ_agg, μ_implikasi)

    total_area = np.sum(μ_agg)
    if total_area == 0: return 0.0, μ_agg
    crisp = np.sum(X_UNIVERSE * μ_agg) / total_area
    return crisp, μ_agg

# Interpretasi nilai crisp ke kategori predikat
def interpretasi(nilai):
    if nilai >= 75.0:
        return "Cum Laude"
    elif nilai >= 50.0:
        return "Sangat Memuaskan"
    else:
        return "Memuaskan"

# Program utama: input, proses, dan visualisasi hasil
if __name__ == "__main__":
    try:
        ipk = float(input("Masukkan IPK mahasiswa (0.0 - 4.0): "))
        publikasi = float(input("Masukkan jumlah publikasi (0 - 10): "))
    except ValueError:
        print("Input tidak valid. Harap masukkan angka.")
        exit()

    rules = inferensi(ipk, publikasi)
    hasil, μ_agg_data = defuzzifikasi_with_plot_data(rules)
    predikat = interpretasi(hasil)

    print("\n===== HASIL PENILAIAN MAHASISWA FUZZY =====")
    print(f"IPK: {ipk}")
    print(f"Publikasi: {int(publikasi)}")
    print("-" * 45)
    print(f"Nilai Crisp Output (Centroid): {hasil:.2f}")
    print(f"Predikat Kelulusan: {predikat} 🎉")
    print("=" * 45)

    if np.sum(μ_agg_data) > 0:
        plt.figure(figsize=(10, 5))
        plt.plot(X_UNIVERSE, μ_agg_data, 'b', linewidth=2, label='Fungsi Agregasi (μ_agg)')
        plt.axvline(x=hasil, color='r', linestyle='--', label=f'Centroid: {hasil:.2f}')
        plt.plot(X_UNIVERSE, MU_MEMUASKAN, 'g:', label='Memuaskan')
        plt.plot(X_UNIVERSE, MU_SANGAT, 'y:', label='Sangat Memuaskan')
        plt.plot(X_UNIVERSE, MU_CUMLAUDE, 'r:', label='Cum Laude')
        plt.title(f'Hasil Inferensi dan Defuzzifikasi Centroid (IPK: {ipk}, Pub: {int(publikasi)})')
        plt.xlabel('Nilai Predikat (0 - 100)')
        plt.ylabel('Derajat Keanggotaan')
        plt.legend(loc='upper left')
        plt.grid(True)
        plt.ylim(0, 1.1)
        plt.show()
